<h2 style='text-align:center'>Create Final Model</h2>

<hr>

In [1]:
# imprt libraries
import os
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
import pickle

In [2]:
#Data Preprocessing

def preprocess(dataset:pd.DataFrame) -> pd.DataFrame :
    '''
    dataset contain this columns:
        'gender':               ['Female' 'Male']
        'SeniorCitizen':        [0 1]
        'Partner':              ['Yes' 'No']
        'Dependents':           ['No' 'Yes']
        'tenure':               int
        'PhoneService':         ['No' 'Yes']
        'MultipleLines':        ['No phone service' 'No' 'Yes']
        'InternetService':      ['DSL' 'Fiber optic' 'No']
        'OnlineSecurity':       ['No' 'Yes' 'No internet service']
        'OnlineBackup':         ['Yes' 'No' 'No internet service']
        'DeviceProtection':     ['No' 'Yes' 'No internet service']
        'TechSupport':          ['No' 'Yes' 'No internet service']
        'StreamingTV':          ['No' 'Yes' 'No internet service']
        'StreamingMovies':      ['No' 'Yes' 'No internet service']
        'Contract':             ['Month-to-month' 'One year' 'Two year']
        'PaperlessBilling':     ['Yes' 'No']
        'PaymentMethod':        ['Electronic check' 'Mailed check' 'Bank transfer (automatic)' 'Credit card (automatic)']
        'MonthlyCharges':       float
        'TotalCharges':         float
        'Churn':                ['No' 'Yes']   // Not necessary
    '''
    df = dataset.copy()

    # Prepare string variables
    df.columns = df.columns.str.lower().str.replace(' ','_')
    string_columns = list(df.dtypes[df.dtypes=='O'].index)
    for col in string_columns:
        df[col] = df[col].str.lower().str.replace(' ','_')

    # drop customerid
    if 'customerid' in df.columns:
        del df['customerid']
        
    # Prepare numeric columns type
    if 'tenure' in df.columns:
        df['tenure'] = pd.to_numeric(df['tenure'], errors='coerce')
        df['tenure'] = df['tenure'].fillna(0)
    else:
        raise Exception("!!! Tenure column didn't exist !!!")

    if 'monthlycharges' in df.columns:
        df['monthlycharges'] = pd.to_numeric(df['monthlycharges'], errors='coerce')
        df['monthlycharges'] = df['monthlycharges'].fillna(0)
    else:
        raise Exception("!!! MonthlyCharges column didn't exist !!!")

    if 'totalcharges' in df.columns:
        df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')
        df['totalcharges'] = df['totalcharges'].fillna(0)
    else:
        raise Exception("!!! TotalCharges column didn't exist !!!")
        
    # Make dependent variable numeric
    if 'churn' in df.columns:
        df.churn = (df.churn == 'yes').astype(int)
    
    return df

In [3]:
# Feature Engineering

def FeatureEngineering(dataset:pd.DataFrame,
                       categorical_variables:list,
                       numerical_variables:list,
                       isTrain:bool,
                       scaleColumns:list):
    """
    dataset: pd.Dataframe
    categorical_variables: list of important categorical columns that we want to use in pur model
    numerical_variables: list of important numerical columns that we want to use in pur model
    isTrain:    if you want to train a model, set it 'True'
                if you want to use for test or deployment set it 'False'
    """
    df = dataset.copy()
    categorical_important = categorical_variables.copy()
    numerical_important = numerical_variables.copy()
    df = df[numerical_important + categorical_important]

    # we create a folder to save our data there 
    path = os.getcwd()
    assets_path = path+'\\assets'
    if not os.path.exists(assets_path):
        os.makedirs(assets_path)

    # Dummy variables
    if isTrain:
        def ohe(dataset:pd.DataFrame, categorical_columns:str):
            df = dataset.copy()
            df_info = pd.DataFrame(columns=['col', 'uniques'])
            for col in categorical_columns:
                list_Uniques = df[col].unique()
                df_info.loc[len(df_info)] = [col, ','.join(map(str, list_Uniques))]
                for name in list_Uniques:
                    df[str(col)+'_'+str(name)] = df[col].apply(lambda x : 1 if x==name else 0)
                del df[col]

            df_info.to_csv(assets_path+'\\ohe_info.csv', index=False)
            return df

        df=ohe(df, categorical_important)
    else:
        df_info = pd.read_csv(assets_path+'\ohe_info.csv')
        for i in range(len(df_info)):
            col = df_info.iloc[i,:].col
            list_Uniques = df_info.iloc[i,:].uniques.split(',')
            for name in list_Uniques:
                df[str(col)+'_'+str(name)] =  df[col].apply(lambda x : 1 if x==name else 0)
            del df[col]

    # Feature scaling
    if isTrain:
        scale_info = pd.DataFrame(columns=['col','mean','std'])
        for col in scaleColumns:
            mean_col = df[col].mean()
            std_col = df[col].std()
            df[col] = (df[col]-mean_col)/std_col
            scale_info.loc[len(scale_info)] = [col, mean_col, std_col]
        
        scale_info.to_csv(assets_path+'\\scale_info.csv', index=False)
    else:
        scale_info = pd.read_csv(assets_path+'\\scale_info.csv')
        
        for i in range(len(scale_info)):
            col = scale_info.loc[i,'col']
            mean_col = scale_info.loc[i,'mean']
            std_col = scale_info.loc[i,'std']

            df[col] = (df[col]-mean_col)/std_col
    return df

In [4]:
df = pd.read_csv('Churn_prediction.csv')
categorical_important=['contract', 'onlinesecurity', 'techsupport', 'internetservice']
numerical_important = ['tenure', 'monthlycharges', 'totalcharges']

y_train = df.Churn
df = preprocess(df)
df = FeatureEngineering(df,
                        categorical_variables=categorical_important,
                        numerical_variables=numerical_important,
                        isTrain=True,
                        scaleColumns=numerical_important)

clf_NB = GaussianNB(var_smoothing = 0.718593).fit( df.loc[:,df.columns!='churn'], y_train )

In [5]:
# Save the model
path = os.getcwd()
assets_path = path+'\\assets'
if not os.path.exists(assets_path):
    os.makedirs(assets_path)

filename = assets_path+'\\Model.sav'
pickle.dump(clf_NB, open(filename, 'wb'))
print('Model successfully created.')

Model successfully created.


<hr>

Use the model 

In [6]:
# imprt libraries
import os
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
import pickle

Customer1 = pd.DataFrame(data={
    'contract': ['Month-to-month'],
    'onlinesecurity': ['Yes'],
    'techsupport': ['No'],
    'internetservice': ['DSL'],
    'tenure': [2],
    'monthlycharges': [53.85],
    'totalcharges': [108.15]})

categorical_important=['contract', 'onlinesecurity', 'techsupport', 'internetservice']
numerical_important = ['tenure', 'monthlycharges', 'totalcharges']

Customer1 = preprocess(Customer1)
Customer1 = FeatureEngineering(Customer1,
                               categorical_variables=categorical_important,
                               numerical_variables=numerical_important,
                               isTrain=False,
                               scaleColumns=numerical_important)

path = os.getcwd()
assets_path = path+'\\assets'
filename = assets_path+'\\Model.sav'
model = pickle.load(open(filename, 'rb'))

pred = model.predict(Customer1)
pred

array(['Yes'], dtype='<U3')

Eventually our model is ready to use :)